<a href="https://colab.research.google.com/github/nerdk312/60_Days_RL_Challenge/blob/master/CURL_Pacman_All_Agents_060520.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
'''

# Used to save Pacman video

In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [1]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!apt-get install ffmpeg

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (46.1.3)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


# Installation and Imports

In [16]:
#!pip install git+git://github.com/mila-iqia/atari-representation-learning.git
!pip install git+git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail
!pip install git+git://github.com/openai/baselines
!pip install wandb

  Cloning git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail to /tmp/pip-req-build-ysgchhmh
  Running command git clone -q git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail /tmp/pip-req-build-ysgchhmh
  Created wheel for a2c-ppo-acktr: filename=a2c_ppo_acktr-0.0.1-cp36-none-any.whl size=18833 sha256=6e4481c8967d495bae5c28ba9e226c96ad67a76d812df711af299ce48d4f0c43
  Stored in directory: /tmp/pip-ephem-wheel-cache-afl3jfzw/wheels/91/52/02/ec5c530fd76d56a66934ee91abbdae5240b766be1dc176deb7
Successfully built a2c-ppo-acktr
  Cloning git://github.com/openai/baselines to /tmp/pip-req-build-x_euv77_
  Running command git clone -q git://github.com/openai/baselines /tmp/pip-req-build-x_euv77_
  Created wheel for baselines: filename=baselines-0.1.6-cp36-none-any.whl size=220664 sha256=a7fc28558ae75931b4bf5acf1931bc2890a2f94dbfc2ceb537f36e84e544b256
  Stored in directory: /tmp/pip-ephem-wheel-cache-5tnyfwax/wheels/42/1c/91/28314e0cd1d2cc57cf8dd18b20c4c9a0f39ae518adc13caf24
Successfully

In [0]:
import os
from __future__ import print_function
import pickle
import sys
sys.path.append('/content/drive/My Drive/Unsupervised_state_representation/atariari')

import wandb

import random
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import datetime
import time

from benchmark.envs import *
from torch.autograd import Variable

import gym
from benchmark.wrapper import AtariARIWrapper

#from benchmark import *
#from methods import utils
# Needed to create dataloaders
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from custom.EarlyStopping import EarlyStopping_loss
from custom.model import *
from custom.LSTM_model import *
from custom.custom_wrappers import DeterminsticNoopResetEnv 
from baselines.common.atari_wrappers import EpisodicLifeEnv#, WarpFrame

# Required for CURL 
from collections import deque
from skimage.util.shape import view_as_windows

In [18]:
!wandb login #############

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


# Utils

In [0]:
def make_dir(dir_path):
    try:
        os.mkdir(dir_path)
    except OSError:
        pass
    return dir_path

class ReplayBuffer(Dataset):
    """Buffer to store environment transitions."""
    def __init__(self, obs_shape, state_shape,action_shape,capacity, batch_size, device,image_size=84,transform=None):
        self.capacity = capacity
        self.batch_size = batch_size
        self.device = device
        self.image_size = image_size
        self.transform = transform

        obs_dtype = np.uint8 # Need to consider the sign of this
        self.obses = np.empty((capacity, *obs_shape), dtype=obs_dtype)
        self.next_obses = np.empty((capacity, *obs_shape), dtype=obs_dtype)
        self.actions = np.empty((capacity,*action_shape),dtype = np.float32)
        self.states = np.empty((capacity, *state_shape), dtype = np.float32) # need to have at least int16 or float32 in order to obtain signed values
        self.next_states = np.empty((capacity, *state_shape),dtype = np.float32) 

        self.idx = 0
        self.last_save = 0
        self.full = False

    def add(self, obs, next_obs,action, state, next_state): # Nawid- Add information to replay buffer
        
        np.copyto(self.obses[self.idx], obs)
        np.copyto(self.actions[self.idx], action)
        np.copyto(self.next_obses[self.idx], next_obs)
        np.copyto(self.states[self.idx], state)
        np.copyto(self.next_states[self.idx],next_state)

        self.idx = (self.idx + 1) % self.capacity # This makes the data gets replaced in a recursive manner when the capacity is full
        self.full = self.full or self.idx == 0
    
    def sample_cpc(self): # Nawid - samples images I believe

        start = time.time()
        idxs = np.random.randint(
            0, self.capacity if self.full else self.idx, size=self.batch_size
        ) # Used to randomly sample indices

        obses = self.obses[idxs] # Nawid - Samples observation
        next_obses = self.next_obses[idxs] # Nawid - Samples next observation
        pos = obses.copy() # Nawid -

        # Nawid - Crop images randomly
        obses = random_crop(obses, self.image_size)
        next_obses = random_crop(next_obses, self.image_size)
        pos = random_crop(pos, self.image_size)

        obses = torch.as_tensor(obses, device=self.device).float()
        
        next_obses = torch.as_tensor(
            next_obses, device=self.device
        ).float()

        actions = torch.as_tensor(self.actions[idxs], device=self.device)

        
        states = torch.as_tensor(self.states[idxs], device=self.device)
        next_states = torch.as_tensor(self.next_states[idxs],device=self.device)
        y_target_position = self.next_states[idxs] - self.states[idxs]
        y_target_position = torch.as_tensor(y_target_position, device=self.device) 

        
        pos = torch.as_tensor(pos, device=self.device).float()
        cpc_kwargs = dict(obs_anchor=obses, obs_pos=pos,
                          time_anchor=None, time_pos=None) # Nawid  Postitive example is pos whilst anchor is obses

        return obses, actions, next_obses, y_target_position, cpc_kwargs
    
    def save(self, save_dir):
        path = os.path.join(save_dir, '%d_%d.pt' % (self.last_save, self.capacity))
        payload = [
            self.obses[0:self.capacity], #  Changed the payload compared to their training as I intend to use a offline training at the moment
            self.next_obses[0:self.capacity],
            self.actions[0:self.capacity],
            self.states[0:self.capacity],
            self.next_states[0:self.capacity]
        ]
        torch.save(payload, path)
    
    def load(self, save_dir): # Nawid - Loads the data into the replay buffer
        chunks = os.listdir(save_dir)
        chucks = sorted(chunks, key=lambda x: int(x.split('_')[0]))
        for chunk in chucks:
            start, end = [int(x) for x in chunk.split('.')[0].split('_')]
            path = os.path.join(save_dir, chunk)
            payload = torch.load(path)
            assert self.idx == start
            self.obses[start:end] = payload[0]
            self.next_obses[start:end] = payload[1]
            self.actions[start:end] = payload[2]
            self.states[start:end] = payload[3]
            self.next_states[start:end] = payload[4]
            self.idx = end

    
    
    def __getitem__(self, idx): # Nawid - Obtains item from replay buffer
        idx = np.random.randint(
            0, self.capacity if self.full else self.idx, size=1
        )
        idx = idx[0]
        obs = self.obses[idx]
        action = self.actions[idx]
        reward = self.rewards[idx]
        next_obs = self.next_obses[idx]
        not_done = self.not_dones[idx]

        if self.transform:
            obs = self.transform(obs)
            next_obs = self.transform(next_obs)

        return obs, action, reward, next_obs, not_done
    
    def __len__(self):
        return self.capacity
    

class FrameStack(gym.Wrapper): # Nawid - Stacks the frames
    def __init__(self, env, k):
        gym.Wrapper.__init__(self, env)
        self._k = k
        self._frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = gym.spaces.Box(
            low=0,
            high=255,
            shape = (shp[:-1]+(shp[-1]*k,)),
            #shape=((shp[0] * k,) + shp[1:]),
            dtype=env.observation_space.dtype
        )
        self._max_episode_steps = 100000 #env._max_episode_steps - could not access max episode steps if i use the warp wrapper so I found the max episode steps manually and placed it in the code

    def reset(self):
        obs = self.env.reset()
        for _ in range(self._k):
            self._frames.append(obs)
        return self._get_obs()

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self._frames.append(obs)
        return self._get_obs(), reward, done, info

    def _get_obs(self):
        assert len(self._frames) == self._k
        return np.concatenate(list(self._frames), axis=2)
    

class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env, width=84, height=84, grayscale=False, dict_space_key=None):
        """
        Warp frames to 84x84 as done in the Nature paper and later work.
        If the environment uses dictionary observations, `dict_space_key` can be specified which indicates which
        observation should be warped.
        """
        super().__init__(env)
        self._width = width
        self._height = height
        self._grayscale = grayscale
        self._key = dict_space_key
        if self._grayscale:
            num_colors = 1
        else:
            num_colors = 3

        new_space = gym.spaces.Box(
            low=0,
            high=255,
            shape=(self._height, self._width, num_colors),
            dtype=np.uint8,
        )
        if self._key is None:
            original_space = self.observation_space
            self.observation_space = new_space
        else:
            original_space = self.observation_space.spaces[self._key]
            self.observation_space.spaces[self._key] = new_space
        assert original_space.dtype == np.uint8 and len(original_space.shape) == 3

    def observation(self, obs):
        if self._key is None:
            frame = obs
        else:
            frame = obs[self._key]

        if self._grayscale:
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(
            frame, (self._width, self._height), interpolation=cv2.INTER_AREA
        )
        if self._grayscale:
            frame = np.expand_dims(frame, -1)

        if self._key is None:
            obs = frame
        else:
            obs = obs.copy()
            obs[self._key] = frame
        return obs


def random_crop(imgs, output_size):
    """
    Vectorized way to do random crop using sliding windows
    and picking out random ones

    args:
        imgs, batch images with shape (B,C,H,W)  -  Changed this to batch images with shape (B,H,W,C)
    """
    # batch size
    n = imgs.shape[0]
    img_size = imgs.shape[1] # Nawid - Changed this to get image size
    crop_max = img_size - output_size
    #imgs = np.transpose(imgs, (0, 2, 3, 1)) #  Nawid - No longer required as I am using a different input size
    w1 = np.random.randint(0, crop_max, n)
    h1 = np.random.randint(0, crop_max, n)
    # creates all sliding windows combinations of size (output_size)
    windows = view_as_windows(
        imgs, (1, output_size, output_size, 1))[..., 0,:,:, 0]
    # selects a random window for each batch element
    cropped_imgs = windows[np.arange(n), w1, h1]
    return cropped_imgs # Nawid -Output shape is the required pytorch shape (channels in dim 1) (B,C,H,W)

def center_crop_image(image, output_size): # Nawid-  Currently not used, used for input for the actor network when choosing an action
    h, w = image.shape[1:]
    print('h,w:', h, w)
    new_h, new_w = output_size, output_size

    top = (h - new_h)//2
    left = (w - new_w)//2

    image = image[:, top:top + new_h, left:left + new_w]
    return image    


def soft_update_params(net, target_net, tau): # Nawid-update for the momentum encoder
    for param, target_param in zip(net.parameters(), target_net.parameters()):
        target_param.data.copy_(
            tau * param.data + (1 - tau) * target_param.data
        )

# Encoder

In [0]:
# for 84 x 84 inputs
OUT_DIM = {2: 39, 4: 35, 6: 31}
# for 64 x 64 inputs
OUT_DIM_64 = {2: 29, 4: 25, 6: 21}

def tie_weights(src, trg): # Nawid - Used to copy weights which is used for conv layes
    assert type(src) == type(trg)
    trg.weight = src.weight
    trg.bias = src.bias

class PixelEncoder(nn.Module):
    """Convolutional encoder of pixels observations."""
    def __init__(self, obs_shape, feature_dim, num_layers=2, num_filters=32,output_logits=False):
        super().__init__()
        assert len(obs_shape) == 3
        self.obs_shape = obs_shape
        self.feature_dim = feature_dim
        self.num_layers = num_layers
    
        self.convs = nn.ModuleList( # Nawid - 3 conv layers in total with output dim 32
            [nn.Conv2d(obs_shape[2], num_filters, 3, stride=2)] # Nawid - Changed this to take into account original numpy ordering
        )
        for i in range(num_layers - 1):
            self.convs.append(nn.Conv2d(num_filters, num_filters, 3, stride=1))

        out_dim = OUT_DIM_64[num_layers] if obs_shape[-2] == 64 else OUT_DIM[num_layers] # changed obs_shape[1] to -2 to take into account numpy ordering
        self.fc = nn.Linear(num_filters * out_dim * out_dim, self.feature_dim)
        self.ln = nn.LayerNorm(self.feature_dim) # Nawid -  Layer norm used instead of batch norm due to statistic sharing

        self.outputs = dict() # Nawid - Saving the outputs are for logging purposes it seems
        self.output_logits = output_logits

    def forward_conv(self, obs):
        obs = obs / 255.
        self.outputs['obs'] = obs
        print('obs:',obs.size())

        conv = torch.relu(self.convs[0](obs))
        self.outputs['conv1'] = conv

        for i in range(1, self.num_layers):
            conv = torch.relu(self.convs[i](conv))
            self.outputs['conv%s' % (i + 1)] = conv
        print('conv:',conv.size())

        h = conv.view(conv.size(0), -1)
        return h
    
    def forward(self, obs, detach=False):
        h = self.forward_conv(obs) # Nawid - Performs conv layers

        if detach:
            h = h.detach()

        h_fc = self.fc(h)
        self.outputs['fc'] = h_fc

        h_norm = self.ln(h_fc) # Nawid - Passes through layer normalisation 
        self.outputs['ln'] = h_norm

        if self.output_logits:
            out = h_norm
        else:
            out = torch.tanh(h_norm)
            self.outputs['tanh'] = out

        return out

    def copy_conv_weights_from(self, source):
        """Tie convolutional layers"""
        # only tie conv layers
        for i in range(self.num_layers):
            tie_weights(src=source.convs[i], trg=self.convs[i])


def make_encoder(obs_shape, feature_dim, num_layers, num_filters, output_logits=False):
    return PixelEncoder(obs_shape, feature_dim, num_layers, num_filters, output_logits)

# CURL

In [0]:
def weight_init(m):
    """Custom weight init for Conv2D and Linear layers."""
    if isinstance(m, nn.Linear): # Nawid-  Weight init for linear layers
        nn.init.orthogonal_(m.weight.data)
        m.bias.data.fill_(0.0)
    elif isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d): # Nawid- weight inti for conv
        # delta-orthogonal init from https://arxiv.org/pdf/1806.05393.pdf
        assert m.weight.size(2) == m.weight.size(3)
        m.weight.data.fill_(0.0)
        m.bias.data.fill_(0.0)
        mid = m.weight.size(2) // 2
        gain = nn.init.calculate_gain('relu')
        nn.init.orthogonal_(m.weight.data[:, :, mid, mid], gain)


class CURL(nn.Module): # Nawid - Module for the contrastive loss
    """
    CURL
    """
    def __init__(self, obs_shape, z_dim, encoder_feature_dim, num_layers, num_filters):
        super(CURL, self).__init__()

        # Need to fix the encoders since I do not plan to use the critics
        self.encoder = make_encoder( # Nawid - Encoder of critic which is also used for the contrastive loss
            obs_shape, encoder_feature_dim, num_layers,
            num_filters, output_logits=True)

        self.encoder_target = make_encoder( obs_shape, encoder_feature_dim, num_layers, 
                                           num_filters, output_logits=True)
        
        self.encoder_target.load_state_dict(self.encoder.state_dict()) # copies the parameters of the encoder into the target encoder which is changing slowly
        self.W = nn.Parameter(torch.rand(z_dim, z_dim)) # Nawid - weight vector for the bilinear product

    def encode(self, x, detach=False, ema=False):
        """
        Encoder: z_t = e(x_t)
        :param x: x_t, x y coordinates
        :return: z_t, value in r2
        """
        if ema:
            with torch.no_grad():
                z_out = self.encoder_target(x)
        else:
            z_out = self.encoder(x)

        if detach:
            z_out = z_out.detach()
        return z_out
    
    def compute_logits(self, z_a, z_pos): # Nawid -  computes logits for contrastive loss
        """
        Uses logits trick for CURL:
        - compute (B,B) matrix z_a (W z_pos.T)
        - positives are all diagonal elements
        - negatives are all other elements
        - to compute loss use multiclass cross entropy with identity matrix for labels
        """
        Wz = torch.matmul(self.W, z_pos.T)  # (z_dim,B)
        logits = torch.matmul(z_a, Wz)  # (B,B)
        logits = logits - torch.max(logits, 1)[0][:, None]
        return logits

class Dynamics_model(nn.Module):
    ''' MLP network'''
    def __init__(self, obs_shape,hidden_dim,
                 output_dim , encoder_feature_dim, num_layers, num_filters):
        super(Dynamics_model,self).__init__()

        self.encoder = make_encoder(obs_shape, encoder_feature_dim,num_layers,num_filters,output_logits=True)

        self.trunk = nn.Sequential(
            nn.Linear(self.encoder.feature_dim + 4 , hidden_dim), nn.ReLU(), # Size of the input is related to the encoder output as well as the concatenated one hot vector for the actions
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
        self.apply(weight_init)
    
    def forward(self, obs,aux, detach_encoder = False):
        obs = self.encoder(obs, detach = detach_encoder) 
        obs = torch.cat((obs, aux), 1) # Join vectors along this dimension
        prediction = self.trunk(obs)        
        return prediction

class CurlAgent(object):
    ''' CURL representation learning'''
    def __init__(
        self,
        obs_shape,
        device,
        hidden_dim = 256,
        output_dim = 2,
        encoder_feature_dim = 50,
        encoder_lr = 1e-3,
        dynamics_lr = 1e-4,
        encoder_tau = 0.005,
        num_layers=4,
        num_filters = 32,
        cpc_update_freq=1,
        encoder_update_freq = 2,
        dynamics_update_freq = 10,
        eval_freq = 100,
        detach_encoder=False,
    ):
        self.device = device
        self.cpc_update_freq = cpc_update_freq
        self.dynamics_update_freq = dynamics_update_freq
        self.eval_freq =  eval_freq
        self.image_size = obs_shape[-2] # Changed this to the numpy dimension
        self.detach_encoder = detach_encoder
        self.encoder_update_freq = encoder_update_freq
        self.encoder_tau = encoder_tau
        
        self.CURL = CURL(obs_shape, encoder_feature_dim,
                         encoder_feature_dim, num_layers, num_filters).to(self.device)
        
        self.Model = Dynamics_model(obs_shape,hidden_dim,output_dim,encoder_feature_dim,num_layers,num_filters) 
        
        # tie encoders between and CURL and dynamics predictor - VERY IMPORTANT-  CAN REMOVE IF I WANT SEPARATE NETWORKS
        self.Model.encoder.copy_conv_weights_from(self.CURL.encoder)

        self.cpc_optimizer = torch.optim.Adam(
                self.CURL.parameters(), lr=encoder_lr
            )
        
        self.dynamics_optimizer = torch.optim.Adam(self.Model.parameters(), lr= dynamics_lr)
        
        self.cross_entropy_loss = nn.CrossEntropyLoss()
        self.MSE_loss = nn.MSELoss() # Nawid - Added this loss for the prediction
        self.train()

        wandb.watch(self.Model, log='all')
        wandb.watch(self.CURL, log='all')
    
    def train(self, training = True):
        self.training = training
        self.CURL.train(training)
        self.Model.train(training)

    def update_cpc(self, obs_anchor, obs_pos, cpc_kwargs):
        z_a = self.CURL.encode(obs_anchor) # Nawid -  Encode the anchor
        z_pos = self.CURL.encode(obs_pos, ema=True) # Nawid- Encode the positive with the momentum encoder

        logits = self.CURL.compute_logits(z_a, z_pos) #  Nawid- Compute the logits between them
        labels = torch.arange(logits.shape[0]).long().to(self.device)
        loss = self.cross_entropy_loss(logits, labels)
        wandb.log({'Contrastive loss':loss})

        self.cpc_optimizer.zero_grad()
        loss.backward()

        self.cpc_optimizer.step()  # Nawid - Used to update the cpc
    
    def update_dynamics(self,obs,actions,labels): # Need to complete this 
        labels = labels/4 # Divided by 4 as 4 is the max predictions which can be made
        prediction = self.Model(obs,actions,detach_encoder=True) # gradient not backpropagated to the encoder
        prediction_loss = self.MSE_loss(prediction,labels)
        wandb.log({'Dynamics loss':prediction_loss})

        self.dynamics_optimizer.zero_grad()
        prediction_loss.backward()
        self.dynamics_optimizer.step()


    def update(self, training_replay_buffer,validation_replay_buffer,step):
        obs, actions_one_hot, next_obs,y_target_position, cpc_kwargs = training_replay_buffer.sample_cpc()
        print(obs.shape)
        if step % self.encoder_update_freq == 0:
            soft_update_params(
                self.CURL.encoder, self.CURL.encoder_target,
                self.encoder_tau
            )

        if step % self.cpc_update_freq == 0:
            obs_anchor, obs_pos = cpc_kwargs["obs_anchor"], cpc_kwargs["obs_pos"]
            self.update_cpc(obs_anchor, obs_pos,cpc_kwargs) # Nawid -  Performs the contrastive loss I believe
        
        if step % self.dynamics_update_freq ==0: # NEed to complete this 
            self.update_dynamics(obs,actions_one_hot,y_target_position)
        
        '''
        if step % self.eval_freq == 0:
            val_obs, val_actions_one_hot, val_next_obs, val_y_target_position, val_cpc_kwargs = validation_replay_buffer.sample_cpc()
            self.validation(val_obs, val_actions_one_hot, val_y_target_position)
        '''

    def validation(self,obs,actions,labels):
        self.Model.eval()
        with torch.no_grad():
            target_labels = labels/4 # Divided by 4 as 4 is the max predictions which can be made
            val_prediction = self.Model(obs,actions,detach_encoder=True) # gradient not backpropagated to the encoder
            val_prediction_loss = self.MSE_loss(val_prediction,target_labels)
            print('val loss:',val_prediction_loss)
            wandb.log({'Dynamics  Validation loss':val_prediction_loss})
            val_prediction = val_prediction *4 # rescaling the outputs
            #print('prediction:',val_prediction)
            #print('targets:', labels) # This is the labels before being rescaled

        self.train()
        
def make_agent(obs_shape, device, dict_info):
    return CurlAgent(
        obs_shape = obs_shape,
        device = device,
        encoder_update_freq =dict_info['encoder_update_freq'],
        dynamics_update_freq = dict_info['dynamics_update_freq'],
        eval_freq = dict_info['eval_freq'],
        encoder_feature_dim = dict_info['encoder_feature_dim'],
        encoder_lr = dict_info['encoder_lr'],
        encoder_tau = dict_info['encoder_tau'],
        num_layers = dict_info['num_layers'],
        num_filters = dict_info['num_filters']
    )

# General Functions

In [0]:
def custom_wrapper(env,deterministic_reset=True,episode_life=True, warp=True, grayscale = False, frame_stack=True,ATARI_labels=True):
    if deterministic_reset:
        env = DeterminsticNoopResetEnv(env,66)
    if episode_life:
        env = EpisodicLifeEnv(env)    
    if warp:
        env = WarpFrame(env,height = 100, width = 100, grayscale=grayscale)
    if frame_stack:
        env = FrameStack(env,4)
    if ATARI_labels:
        env = AtariARIWrapper(env) 
    return env

class General_functions():
    def __init__(self, ENV_NAME, n_actions,possible_positions):
        self.ENV_NAME = ENV_NAME
        self.env = gym.make(self.ENV_NAME)
        self.env = custom_wrapper(self.env)
        self.initial_info_labels = self.env.labels()
        self.key_list = ['player_x', 'player_y']
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(self.device)
        #self.key_list = ['enemy_blinky_x', 'enemy_blinky_y','enemy_pinky_x','enemy_pinky_y','enemy_inky_x','enemy_inky_y', 'enemy_sue_x', 'enemy_sue_y', 'player_x', 'player_y']

        self.prev_action_counter = False
        self.repeated_end = False
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.n_actions = n_actions
        # possible positions is numpy array with most of the possible positions that the agent can go to
        self.possible_positions = possible_positions
        self.possible_positions_list = self.possible_positions.tolist()
        
    def one_hot(self,i):
        a = np.zeros(self.n_actions, 'uint8')
        a[i-1] = 1
        return a
    
    def state_conversion(self,info_labels):
        state = [info_labels[word] for word in self.key_list if word in info_labels]                
        state = np.array(state)
        return state

    def next_position(self,state, action):
        next_position = state[-2:].copy()
        #print(next_position)
        if action == 1:
            next_position[1] = next_position[1] - 2 
        elif action == 2:
            next_position[0] = next_position[0] + 2
        elif action == 3:
            next_position[0] = next_position[0] - 2
        elif action == 4:
            next_position[1] = next_position[1] + 2        
        if next_position.tolist() in self.possible_positions_list: # possible positions will be a list which is fed into the network 
            return True
        else:
            return False

    def check_all_agents(self,info_label,next_info_label):
        repeated = np.equal(info_label,next_info_label).all()
        if repeated:
            self.repeated_end= True

    def check_state(self,state, next_state):
        repeated_state = np.equal(state[-2:], next_state[-2:]).all()
        if repeated_state:
            self.prev_action_counter = False

# Data collection and preprocessing

In [0]:
class Data_collection(General_functions):
    def __init__(self, ENV_NAME, n_actions, possible_positions,info_dict):
        super(Data_collection, self).__init__(ENV_NAME, n_actions, possible_positions)
        self.grayscale = info_dict['grayscale']
        self.channels = 1 if self.grayscale else 3 
        self.obs_shape = (info_dict['image_size'], info_dict['image_size'],self.channels*info_dict['frame_stack']) # Nawid - Stack together images (multiply of 3 present due to 3 channels (RGB))
        self.pre_aug_obs_shape = (info_dict['pre_transform_image_size'],info_dict['pre_transform_image_size'],self.channels*info_dict['frame_stack'])
        self.state_shape = (2,)
        self.action_shape = (n_actions,)        
        self.replay_buffer  = ReplayBuffer(self.pre_aug_obs_shape,self.state_shape,self.action_shape,info_dict['replay_buffer_capacity'], info_dict['batch_size'],self.device, info_dict['image_size'])     #(self, obs_shape, capacity, batch_size, device,image_size=84,transform=None):


    def random_action_selection(self, state,prev_action = None):
        while True:
            action = np.random.randint(1,5) 
            feasible_action = self.next_position(state,action)
            if feasible_action:
                self.prev_action_counter = True
                return action, None
            else:
                infeasible_action_one_hot = self.one_hot(action)
                if self.prev_action_counter and prev_action !=None:
                    return prev_action, infeasible_action_one_hot 
    
    def gather_random_trajectories(self,num_traj):        
        ts = time.gmtime()
        ts = time.strftime('%d-%m_%H:%M')
        work_dir = '/content/drive/My Drive/MsPacman-data' + '/' + ts
        work_dir = make_dir(work_dir)
        buffer_dir = make_dir(os.path.join(work_dir, 'buffer'))
        for n in range(num_traj):
            
            if n % 10 ==0:
                print('trajectory number:',n)
                # Initial set up
            #self.env.seed(0)


            self.env = gym.make(self.ENV_NAME) # Due to error in code, I reinstantiate the env each time
            self.env = custom_wrapper(self.env,grayscale = self.grayscale)
            obs = self.env.reset()
            

            self.repeated_end = False
            info_labels = self.env.labels() # Nawid - Used to get the current state
            state = self.state_conversion(info_labels) # Used to get the initial state
            prev_action = None # Initialise prev action has having no action
            

            while True:
                sampled_action, infeasible_action_one_hot = self.random_action_selection(state,prev_action)
                sampled_action_one_hot = self.one_hot(sampled_action)
                
                next_obs, reward, done, next_info = self.env.step(sampled_action)
                next_info_labels = next_info['labels']
                
                next_state = self.state_conversion(next_info_labels)
                
                self.check_state(state,next_state)
                self.check_all_agents(info_labels, next_info_labels) # need to use the info labels to predict the state as the info labels have all the informaiton
                    
                if not self.repeated_end:
                    
                    if infeasible_action_one_hot is not None:
                        fake_next_state = np.zeros_like(state) #  Need to instantiate a new version each time to prevent updating a single variable which will affect all places(eg lists) where the variable is added
                        fake_next_state[0:-2] = next_state[0:-2].copy() # the enemy position of the fake next state is the current enemy position
                        fake_next_state[-2:] = state[-2:].copy() # The agent position for the fake next state is the state before any action was taken

                        self.replay_buffer.add(obs, next_obs, infeasible_action_one_hot, state, fake_next_state)
                        
                    
                    self.replay_buffer.add(obs,next_obs, sampled_action_one_hot, state, next_state)
                else:
                    done = True   
                
                obs = next_obs # do not need to copy as a new variable of obs is instantiated at each time step.
                state = next_state.copy()
                info_labels = next_info_labels.copy()
                prev_action = sampled_action

                if done:
                    break    

                if self.replay_buffer.full:
                    self.replay_buffer.save(buffer_dir)
                    return

# Hyperparameters

In [24]:
ENV_NAME = 'MsPacmanDeterministic-v4'
n_actions = 4 #9 - Nawid - Change to 5 actions as the 4 other actions are simply copies of the other actions, therefore 5 actions should lower the amount of data needed.
parse_dict= {'pre_transform_image_size':100,
             'image_size':84,
             'action_repeat':1,
             'frame_stack':4,
             'replay_buffer_capacity':10000,
             'agent':'curl_agent',
             'num_train_steps':10000,
             'batch_size':1024,
             'encoder_update_freq':2,
             'dynamics_update_freq':10,
             'eval_freq': 100,
             'encoder_feature_dim':50,
             'encoder_lr':1e-3,
             'dynamics_lr':1e-4,
             'encoder_tau':0.05,
             'num_layers':4,
             'num_filters':32,
             'grayscale': True
            }


possible_positions = np.load('/content/drive/My Drive/MsPacman-data/possible_pacman_positions.npy',allow_pickle=True)
load_trajectories = True
preloaded_train_data = '/content/drive/My Drive/MsPacman-data/08-05_16:21/buffer'
preloaded_val_data = '/content/drive/My Drive/MsPacman-data/08-05_17:13/buffer'
pretrain_model = False
wandb.init(entity="nerdk312", project="Contrastive_learning",config = parse_dict)

config = wandb.config
if load_trajectories:
    config.loaded_train_trajectories = preloaded_train_data
    config.loaded_val_trajectories = preloaded_val_data

if pretrain_model:
    config.pretrained_model = pretrain_model_dir

# Main -  Model training

Data - Collection

In [25]:
data_object = Data_collection(ENV_NAME,n_actions,possible_positions, parse_dict)
val_data_object = Data_collection(ENV_NAME,n_actions,possible_positions, parse_dict)
if load_trajectories:
    data_object.replay_buffer.load(preloaded_train_data)
    val_data_object.replay_buffer.load(preloaded_val_data)
    
else:
    data_object.gather_random_trajectories(5000)
    val_data_object.gather_random_trajectories(5000)

cpu
cpu


In [27]:
agent = make_agent(
        obs_shape = data_object.obs_shape,
        device =data_object.device,
        dict_info = parse_dict
    )
for step in range(parse_dict['num_train_steps']):
    agent.update(data_object.replay_buffer,val_data_object.replay_buffer,step)



torch.Size([1024, 4, 84, 84])
obs: torch.Size([1024, 4, 84, 84])
conv: torch.Size([1024, 32, 35, 35])
obs: torch.Size([1024, 4, 84, 84])
conv: torch.Size([1024, 32, 35, 35])
obs: torch.Size([1024, 4, 84, 84])
conv: torch.Size([1024, 32, 35, 35])
torch.Size([1024, 4, 84, 84])
obs: torch.Size([1024, 4, 84, 84])
conv: torch.Size([1024, 32, 35, 35])
obs: torch.Size([1024, 4, 84, 84])
conv: torch.Size([1024, 32, 35, 35])


KeyboardInterrupt: ignored

# Main - Model training

Data collection and processing

Dynamics model training

In [0]:
#env_model = NNDynamicsModel(n_actions + 2, 2).to(device) # Nawid - Need to put both actions as it is a one-hot vector
env_model = recurrent_dynamics_model(compulsory_dict['input_dim'], compulsory_dict['hidden_dim'], compulsory_dict['output_dim']).to(device)
if compulsory_dict['pretrain_model']:
    #env_model.load_state_dict(torch.load(pretrained_env_model))
    checkpoint = torch.load(pretrained_env_model)
    env_model.load_state_dict(checkpoint['model_state_dict'])
    
    for parameter in env_model.parameters():
        parameter.requires_grad = False
    env_model.eval()
else:
    env_optimizer = torch.optim.Adam(env_model.parameters(),compulsory_dict['lr']) # Nawid - Optimizer for the env model
    wandb.watch(env_model, log="all")
    env_model_name = 'Env_model'+ '_' + date_time
    early_stopping_env = EarlyStopping_loss(patience=5, verbose=True, wandb=wandb, name=env_model_name)
    for n_iter in range(compulsory_dict['AGGR_ITER']):
        if early_stopping_env.early_stop:
            print('Early stopping')
            break 
        train_recurrent_model(norm_env_train_data, norm_env_val_data, env_model, compulsory_dict['BATCH_SIZE'],compulsory_dict['timesteps'], compulsory_dict['TRAIN_MODEL_ITER'],env_optimizer,device,early_stopping_env)